# Домашнее задание 8

#### КЛАССЫ И ОБЪЕКТНО-ОРИЕНТИРОВАННОЕ ПРОГРАММИРОВАНИЕ

**Задание 1**

Напишите функцию, которая возвращает название валюты (поле 'Name') с максимальным значением курса с помощью сервиса https://www.cbr-xml-daily.ru/daily_json.js

In [215]:
import requests

In [216]:
def max_rate_cur():
    '''
    Возвращает название валюты с максимальным значением курса за единицу
    Если таких несколько (что крайне маловероятно), возвращает первое
    СДР (специальные права заимствования) МВФ валютой не считаются и из анализа исключаются 
    '''
    try:
        rates = requests.get('https://www.cbr-xml-daily.ru/daily_json.js').json()['Valute']
    except:
        print("Ошибка при получении данных.")
        return
    
    only_rates = [rate for rate in rates.values() if rate['CharCode'] != 'XDR'] # excl SDR - a busket of currencies
    
    return max(only_rates, key = lambda x : x['Value'] / x['Nominal'])['Name']
    
max_rate_cur()

'Фунт стерлингов Соединенного королевства'

**Задание 2**

Добавьте в класс Rate параметр diff (со значениями True или False), который в случае значения True в методах eur и usd будет возвращать не курс валюты, а изменение по сравнению в прошлым значением. Считайте, self.diff будет принимать значение True только при возврате значения курса. При отображении всей информации о валюте он не используется.

In [217]:
import requests

class Rate:
    def __init__(self, format='value', diff = False):
        self.format = format
        self.diff = diff
    
    def exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return r.json()['Valute']
    
    def make_format(self, currency):
        """
        Возвращает информацию о валюте currency в вариантах:
        1. полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        2. информация о курсе на сегодня:
        Rate('value').make_format('EUR')
        79.4966
        
        3. изменение курса за последний день:
        Rate('value', True).make_format('EUR')
        -0.1202
        """
        response = self.exchange_rates()
        
        if currency in response:
            if self.format == 'full':
                return response[currency]
            
            if self.format == 'value':
                if not self.diff:
                    return response[currency]['Value']
                return round(response[currency]['Value'] - response[currency]['Previous'], 4)
        
        return 'Error'
    
    def eur(self):
        """Возвращает курс евро на сегодня в формате self.format"""
        return self.make_format('EUR')
    
    def usd(self):
        """Возвращает курс доллара на сегодня в формате self.format"""
        return self.make_format('USD')

    def AZN(self):
        """Возвращает курс азербайджанского маната на сегодня в формате self.format"""
        return self.make_format('AZN')

In [218]:
print(Rate().usd())
print(Rate(diff = True).usd())

print(Rate().eur())
print(Rate(diff = True).eur())

print(Rate().AZN())
print(Rate('value', True).AZN())

61.5333
-0.0361
68.5358
-0.1202
36.2707
-0.0213


**Задание 3**

Напишите класс Designer, который учитывает количество международных премий. Подсказки в коде занятия в разделе "Домашнее задание задача 3":

    Напишите класс Designer, который учитывает количество международных премий для дизайнеров (из презентации: "Повышение на 1 грейд за каждые 7 баллов. Получение международной премии – это +2 балла"). Считайте, что при выходе на работу сотрудник уже имеет две премии и их количество не меняется со стажем (конечно если хотите это можно вручную менять). Выполните проверку для 20 аккредитаций дизайнера Елены.

In [219]:
class Employee:
    def __init__(self, name, seniority):
        self.name = name
        self.seniority = seniority
        
        self.grade = 1
    
    def grade_up(self):
        """Повышает уровень сотрудника"""
        self.grade += 1
    
    def publish_grade(self):
        """Публикация результатов аккредитации сотрудников"""
        print(self.name, self.grade)
    
    def check_if_it_is_time_for_upgrade(self):
        pass

In [220]:
class Designer(Employee):
    def __init__(self, name, seniority, awards):
        super().__init__(name, seniority)
        self.awards = awards
        self.seniority += awards * 2
    
    def check_if_it_is_time_for_upgrade(self):
        # для каждой аккредитации увеличиваем счетчик на 1
        # пока считаем, что все дизайнеры проходят аккредитацию
        self.seniority += 1
        
        # условие повышения сотрудника из презентации
        if self.seniority % 7 == 0:          # т.к. премии учитываются с шагом 2, если добавить изменение премий, можно пропустить апгрейд
            self.grade_up()
        
        # публикация результатов
        return self.publish_grade()

In [221]:
elena = Designer('Елена', seniority=0, awards=2)

In [222]:
for i in range(20):
    elena.check_if_it_is_time_for_upgrade()

Елена 1
Елена 1
Елена 2
Елена 2
Елена 2
Елена 2
Елена 2
Елена 2
Елена 2
Елена 3
Елена 3
Елена 3
Елена 3
Елена 3
Елена 3
Елена 3
Елена 4
Елена 4
Елена 4
Елена 4


**С возможностью добавления премий**

In [223]:
class Designer(Employee):
    def __init__(self, name, seniority, awards):
        super().__init__(name, seniority)
        self.awards = awards
        self.seniority += awards * 2
    
    def __str__(self):
        # возвращает информацию об экземпляре класса
        return f'Designer with {self.seniority} seniority, {self.awards} awards, {self.grade} grade'
    
    def check_if_it_is_time_for_upgrade(self):
        # условие повышения сотрудника из презентации
        if self.seniority // 7 >= self.grade:          # т.к. премии учитываются с шагом 2
            self.grade_up()
        
        # публикация результатов
        return self.publish_grade()
    
    def new_award(self):
        # для каждой новой международной премии увеличиваем счетчик на 2   
        print('! award !')
        self.awards += 1
        self.seniority += 2
        self.check_if_it_is_time_for_upgrade()
        
    def pass_accreditation(self):
        # для каждой аккредитации увеличиваем счетчик на 1
        # пока считаем, что все дизайнеры проходят аккредитацию
        self.seniority += 1
        self.check_if_it_is_time_for_upgrade()    

In [224]:
import random 

elena = Designer('Елена', seniority=0, awards=2)

for i in range(20):
    elena.pass_accreditation()
    if not random.randint(0, 7):
        elena.new_award()
    print(elena)

Елена 1
Designer with 5 seniority, 2 awards, 1 grade
Елена 1
Designer with 6 seniority, 2 awards, 1 grade
Елена 2
Designer with 7 seniority, 2 awards, 2 grade
Елена 2
Designer with 8 seniority, 2 awards, 2 grade
Елена 2
Designer with 9 seniority, 2 awards, 2 grade
Елена 2
Designer with 10 seniority, 2 awards, 2 grade
Елена 2
Designer with 11 seniority, 2 awards, 2 grade
Елена 2
! award !
Елена 3
Designer with 14 seniority, 3 awards, 3 grade
Елена 3
Designer with 15 seniority, 3 awards, 3 grade
Елена 3
! award !
Елена 3
Designer with 18 seniority, 4 awards, 3 grade
Елена 3
Designer with 19 seniority, 4 awards, 3 grade
Елена 3
Designer with 20 seniority, 4 awards, 3 grade
Елена 4
Designer with 21 seniority, 4 awards, 4 grade
Елена 4
Designer with 22 seniority, 4 awards, 4 grade
Елена 4
Designer with 23 seniority, 4 awards, 4 grade
Елена 4
Designer with 24 seniority, 4 awards, 4 grade
Елена 4
Designer with 25 seniority, 4 awards, 4 grade
Елена 4
Designer with 26 seniority, 4 awards, 4 gra